In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopy
df = pd.read_csv('earthquakes_2023_global.csv')

df['time'] = pd.to_datetime(df['time'].apply(lambda x: x[:-5]))
df['updated'] = pd.to_datetime(df['updated'].apply(lambda x: x[:-5]))
df.drop_duplicates( inplace= True)
# df['hour'] == df['hour'].astype('int8') everything is True I can change
df['hour'] = df['time'].dt.hour.astype('int8')
df['month'] = df['time'].dt.month_name().astype('category')
df[['magType', 'net']] = df[['magType', 'net']].astype('category')

df['nst'].fillna(0, inplace=True) # I took NaN as 0, nobody reported (calculated localy)
df['nst'] = df['nst'].astype('int16') # df['nst'] == df['nst'].astype('int16')

# Too much RAM for these features, I also will calculate errors
df[['latitude', 'longitude', 'depth', 'gap', 'dmin', 'rms']] = \
    df[['latitude', 'longitude', 'depth', 'gap', 'dmin', 'rms']].astype('float32')

''' 
-- After float16 it gives me 3.099609 this kind numbers, if round it I will get back float64 type -- 
df['mag'] = df['mag'].astype('float16').apply(lambda x: round(x, 2)) 
'''
# df['id'] = df['id'].apply(lambda x: x[:2]) --> df['id'] == df['net'] 
df.drop('id', axis = 1, inplace= True)

In [12]:
df.columns

Index(['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'nst',
       'gap', 'dmin', 'rms', 'net', 'updated', 'place', 'type',
       'horizontalError', 'depthError', 'magError', 'magNst', 'status',
       'locationSource', 'magSource', 'hour', 'month'],
      dtype='object')

In [13]:
def clean_place(x):
    
    def check_num(y):
        sigh = False
        for i in y:
            if i.isdigit():
                sigh = True
        return sigh
    
    def taking_km(y):
        y = y.split(' ')
        if any(char.isdigit() for char in y[0]) and any(char.isalpha() for char in y[0]):
            km_till_e = y[0][:-2] + ' ' + y[1]
            reg_from_km = ' '.join(y[3:])
        else:   
            km_till_e = y[0] + ' ' + y[2]
            reg_from_km = ' '.join(y[4:])
        return km_till_e, reg_from_km
    region = None
    km_till_earq = None
    country = None
    
    if ',' in x:
        splitted = x.split(',')
        if len(splitted) == 3:
           country =  splitted[2]
           if check_num(splitted[0]):
                km_till_earq, region = taking_km(splitted[0])
           else:
               region = splitted[1] + ',' + splitted[0]
        if len(splitted) == 2:
            if 'reg' not in splitted[1]:
                country = splitted[1]
            else:
                region = splitted[1]
            if check_num(splitted[0]):
                km_till_earq, region = taking_km(splitted[0])
    else:
        if check_num(x):
            km_till_earq, region = taking_km(x)   
        if 'reg' not in x:
                country = x
        else:
            region = x         
    return region, km_till_earq, country
df['place'].fillna(' ', inplace= True)
df[['region', 'km', 'country']] = df['place'].apply(lambda x: pd.Series(clean_place(x)))
        